In [1]:
using JuMP, Gurobi, CPLEX
using LinearAlgebra

# Problem setting

- 1 vehicle
- Nodes: Depot, 5 Pickup locations (P1 to P5), and 5 Delivery locations (D1 to D5).
- Time Windows: Randomly generated for each pickup and delivery location.
- Stack Limit: 3 (maximum number of items the vehicle can carry).
- Cost Matrix: Randomly generated costs between each pair of locations.

## Gurobi

In [67]:
# Number of pickup and delivery requests
num_requests = 5
num_nodes = 2 * num_requests + 2 # Includes two depot nodes

# Define the set of pickup nodes P and delivery nodes D
P = 1:num_requests
D = num_requests+1:2*num_requests
depot_start = 2*num_requests+1
depot_end = num_nodes
vehicle_capacity = 3

# Define time windows for each node (here, we're setting random examples)
time_windows = [(0, 24)] # Time window for the initial depot
time_windows = vcat(time_windows, [(rand(1:10), rand(11:20)) for _ in P], [(rand(21:30), rand(31:40)) for _ in D])
time_windows = vcat(time_windows, [(30, 40)]) # Time window for the final depot

# Define load sizes for pickup and delivery requests (here, we're setting random examples)
loads = [1 for _ in P] # Random loads for pickups
loads = vcat(loads, -loads) # Loads for deliveries are the negated pickups
loads = vcat([0], loads, [0]) # Loads for depots are zero

# Random service durations at each location
service_durations = rand(1:4, 2*num_requests) # Service durations between 1 to 4 units
service_durations = vcat([0], service_durations, [0])

# Define a random symmetric travel time matrix respecting triangular inequality
# 노드의 무작위 위치 생성
node_positions = [rand(0:10, 2) for _ in 1:num_nodes]

# 유클리드 거리를 사용하여 이동 시간 계산
function calculate_travel_time(pos1, pos2)
    return round(norm(pos1 - pos2)) # 유클리드 거리를 기반으로 이동 시간 계산
end

# 이동 시간 및 대칭 비용 행렬 생성
travel_time_matrix = zeros(num_nodes, num_nodes)
cost_matrix = zeros(num_nodes, num_nodes)
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            travel_time_matrix[i,j] = calculate_travel_time(node_positions[i], node_positions[j])
            cost_matrix[i,j] = travel_time_matrix[i,j] # 비용은 이동 시간에 비례한다고 가정
        end
    end
end

# Define the maximum route duration (here, we're setting a random example)
max_route_duration = rand(50:100)

53

In [60]:
time_windows

12-element Vector{Tuple{Int64, Int64}}:
 (0, 24)
 (3, 13)
 (1, 20)
 (9, 15)
 (2, 15)
 (1, 18)
 (21, 40)
 (26, 37)
 (29, 37)
 (21, 35)
 (21, 39)
 (30, 40)

In [61]:
loads

12-element Vector{Int64}:
  0
  1
  1
  1
  1
  1
 -1
 -1
 -1
 -1
 -1
  0

In [69]:
service_durations

12-element Vector{Int64}:
 0
 4
 3
 1
 3
 3
 1
 1
 3
 1
 4
 0

In [62]:
cost_matrix

12×12 Matrix{Float64}:
 0.0  3.0   7.0   4.0  1.0  1.0  2.0   5.0  3.0   4.0  2.0   6.0
 3.0  0.0   6.0   7.0  4.0  4.0  1.0   4.0  1.0   5.0  3.0   5.0
 7.0  6.0   0.0  10.0  7.0  8.0  7.0  10.0  7.0   4.0  5.0  11.0
 4.0  7.0  10.0   0.0  4.0  4.0  6.0   8.0  7.0   6.0  5.0   9.0
 1.0  4.0   7.0   4.0  0.0  1.0  3.0   6.0  4.0   4.0  2.0   7.0
 1.0  4.0   8.0   4.0  1.0  0.0  3.0   5.0  4.0   5.0  3.0   6.0
 2.0  1.0   7.0   6.0  3.0  3.0  0.0   4.0  1.0   5.0  3.0   4.0
 5.0  4.0  10.0   8.0  6.0  5.0  4.0   0.0  3.0   9.0  7.0   1.0
 3.0  1.0   7.0   7.0  4.0  4.0  1.0   3.0  0.0   6.0  4.0   4.0
 4.0  5.0   4.0   6.0  4.0  5.0  5.0   9.0  6.0   0.0  2.0  10.0
 2.0  3.0   5.0   5.0  2.0  3.0  3.0   7.0  4.0   2.0  0.0   8.0
 6.0  5.0  11.0   9.0  7.0  6.0  4.0   1.0  4.0  10.0  8.0   0.0

In [63]:
travel_time_matrix

12×12 Matrix{Float64}:
 0.0  3.0   7.0   4.0  1.0  1.0  2.0   5.0  3.0   4.0  2.0   6.0
 3.0  0.0   6.0   7.0  4.0  4.0  1.0   4.0  1.0   5.0  3.0   5.0
 7.0  6.0   0.0  10.0  7.0  8.0  7.0  10.0  7.0   4.0  5.0  11.0
 4.0  7.0  10.0   0.0  4.0  4.0  6.0   8.0  7.0   6.0  5.0   9.0
 1.0  4.0   7.0   4.0  0.0  1.0  3.0   6.0  4.0   4.0  2.0   7.0
 1.0  4.0   8.0   4.0  1.0  0.0  3.0   5.0  4.0   5.0  3.0   6.0
 2.0  1.0   7.0   6.0  3.0  3.0  0.0   4.0  1.0   5.0  3.0   4.0
 5.0  4.0  10.0   8.0  6.0  5.0  4.0   0.0  3.0   9.0  7.0   1.0
 3.0  1.0   7.0   7.0  4.0  4.0  1.0   3.0  0.0   6.0  4.0   4.0
 4.0  5.0   4.0   6.0  4.0  5.0  5.0   9.0  6.0   0.0  2.0  10.0
 2.0  3.0   5.0   5.0  2.0  3.0  3.0   7.0  4.0   2.0  0.0   8.0
 6.0  5.0  11.0   9.0  7.0  6.0  4.0   1.0  4.0  10.0  8.0   0.0

In [64]:
max_route_duration

90

In [74]:
# Initialize the model
model = Model(Gurobi.Optimizer)

# Define decision variables
@variable(model, x[1:num_nodes, 1:num_nodes], Bin)
@variable(model, service_time[1:num_nodes] >=0)
@variable(model, vehicle_load[1:num_nodes] >=0)

# Define the objective function to minimize the total cost
@objective(model, Min, sum(cost_matrix[i,j] * x[i,j] for i in 1:num_nodes, j in 1:num_nodes))

# Define the constraints
# Each pickup and delivery match
for i in P
    @constraint(model, sum(x[i, j] for j in num_nodes) == 1)  
    @constraint(model, sum(x[i, j] for j in num_nodes) - sum(x[i+num_requests,j] for j in num_nodes) == 0) 
end

# Leave from and return to the depot
@constraint(model, sum(x[depot_start, j] for j in num_nodes) == 1)  # Leave the depot
@constraint(model, sum(x[i, depot_end] for i in num_nodes) == 1)  # Return to the depot

for i in 1:num_nodes
    @constraint(model, sum(x[i, j] for j in 1:num_nodes) - sum(x[j, i] for j in 1:num_nodes) == 0)
end

# Service time must fall within the time window for each node
for i in 1:num_nodes
    @constraint(model, service_time[i] >= time_windows[i][1])
    @constraint(model, service_time[i] <= time_windows[i][2])
end

# Service time constraints
for i in 1:num_nodes
    for j in 1:num_nodes
        service_time[j] >= (service_time[i] + service_durations[i] + travel_time_matrix[i,j]) * x[i,j]
    end
end

# Vehicle load and capacity constraints
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            @constraint(model, vehicle_load[j] >= (vehicle_load[i] + loads[j]) * x[i,j])
        end
    end
end

for i in 1:num_nodes
    @constraint(model, vehicle_load[i] <= min(vehicle_capacity, vehicle_capacity+loads[i]))
    @constraint(model, vehicle_load[i] >= max(0, loads[i]))
end

# Solve the problem
optimize!(model)

# Check if the model has a solution
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
    println("An optimal solution has been found:")
    
    # Print the route
    println("Vehicle route:")
    for i in 1:num_nodes
        for j in 1:num_nodes
            if value(x[i,j]) > 0.9
                println("Node $i to node $j")
            end
        end
    end
    
    # Print the service start times
    println("\nService start times:")
    for i in 1:2*
        println("Node $i: ", value(service_time[i]))
    end
    
    # Print the vehicle loads
    println("\nVehicle loads after each node:")
    for i in 1:num_nodes
        println("Node $i: ", value(vehicle_load[i]))
    end
else
    println("No optimal solution found. Status: ", termination_status(model))
end


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21


LoadError: MethodError: no method matching isless(::QuadExpr, ::VariableRef)

[0mClosest candidates are:
[0m  isless(::Any, [91m::Missing[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:88[24m[39m
[0m  isless([91m::Missing[39m, ::Any)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:87[24m[39m


In [72]:
min(1,3)

1

In [9]:


# Define time windows for each node (example)
time_windows = [(0,0)] # Depot
time_windows = vcat(time_windows, [(rand(1:10), rand(11:20)) for _ in P], [(rand(21:30), rand(31:40)) for _ in D])
time_windows = vcat(time_windows, [(40,40)]) # Depot

# Define load sizes for pickup and delivery requests
loads = [1 for _ in P] # Assuming a load size of 1 for simplicity
loads = vcat(loads, -loads) # Delivery loads are negative

# Define a symmetric cost matrix
cost_matrix = [rand(1:10) for i in 1:num_nodes, j in 1:num_nodes]
for i in 1:num_nodes
    for j in i:num_nodes
        cost_matrix[i,j] = cost_matrix[j,i] = (i == j) ? 0 : cost_matrix[i,j]
    end
end

# Define a symmetric travel time matrix respecting triangular inequality
travel_time_matrix = cost_matrix # For simplicity, using the same values as cost

# Initialize the model
model = Model(Gurobi.Optimizer)

# Define decision variables
@variable(model, x[1:num_nodes, 1:num_nodes], Bin)
@variable(model, service_time[1:num_nodes])
@variable(model, vehicle_load[1:num_nodes])

# Define the objective function to minimize the total cost
@objective(model, Min, sum(cost_matrix[i,j] * x[i,j] for i in 1:num_nodes, j in 1:num_nodes))

# Constraints
# Each pickup node must be visited exactly once
for i in P
    @constraint(model, sum(x[i,j] for j in [depot; D]) == 1)
end

# Flow conservation for the vehicle
@constraint(model, sum(x[depot, j] for j in P) == 1)
@constraint(model, sum(x[i, depot] for i in D) == 1)

# Service time must fall within the time window for each node
for i in 1:num_nodes
    @constraint(model, service_time[i] >= time_windows[i][1])
    @constraint(model, service_time[i] <= time_windows[i][2])
end

# The vehicle's load must not exceed its capacity after each pickup or delivery
@constraint(model, vehicle_load[depot] == 0)
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            @constraint(model, vehicle_load[j] >= vehicle_load[i] + loads[i] * x[i,j])
        end
    end
end

# Solve the problem
optimize!(model)

# Check if the model has a solution
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
    println("An optimal solution has been found:")
    
    # Print the route
    println("Vehicle route:")
    for i in 1:num_nodes
        for j in 1:num_nodes
            if value(x[i,j]) > 0.9
                println("Node $i to node $j")
            end
        end
    end
    
    # Print the service start times
    println("\nService start times:")
    for i in 1:num_nodes
        println("Node $i: ", value(service_time[i]))
    end
    
    # Print the vehicle loads
    println("\nVehicle loads after each node:")
    for i in 1:num_nodes
        println("Node $i: ", value(vehicle_load[i]))
    end
else
    println("No optimal solution found. Status: ", termination_status(model))
end


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21


LoadError: BoundsError: attempt to access 10-element Vector{Int64} at index [11]

## Dynanic Programming